In [ ]:
import csv
import random
import os
property_label = "P19"
orientation = "objects"

In [ ]:
# load all instances
instances = {}
tsv_file = open(f'../output/allConstraintsAnalysis_Final/instances/instances.{orientation}.{property_label}.tsv', encoding="utf-8")
read_tsv = csv.reader(tsv_file, delimiter="\t")
for line in read_tsv:
    if line[0] not in instances:
        instances[line[0]] = [line[1]]
    else:
        instances[line[0]].append(line[1])
print(len(instances))

In [ ]:
# make file with relevant transE embeddings

transEheader = ['Qnode']
for i in range(0,100):
    transEheader.append("pos"+str(i))
transEheader.append("Yval")

with open(f"../embeddings/profile_graph_embeddings.transE.tsv", encoding="utf-8") as in_file, open(f"../output/instance_prediction_datasets/250000/{property_label}.{orientation}.transE.training.tsv", "w", encoding="utf-8", newline='') as out_file:
    tsv_reader = csv.reader(in_file, delimiter="\t")
    tsv_writer = csv.writer(out_file, delimiter="\t")
    next(tsv_reader)
    tsv_writer.writerow(transEheader)
    for line in tsv_reader:
        if line[0] in instances:
            this_arr = []
            for elem in line:
                this_arr.append(elem)
            # randomly select one of the possible instances to use in the training data
            this_arr.append(random.choice(instances[line[0]]))
            tsv_writer.writerow(this_arr)
            break

In [ ]:
# fit autogluon models
all_data = TabularDataset(f"../output/instance_prediction_datasets/250000/{property_label}.{orientation}.transE.training.tsv")
print(int(len(all_data)*.75))
train_data = all_data[:int(len(all_data)*.75)]
test_data = all_data[int(len(all_data)*.75):]
save_path = f'../embeddings/models/agModels.{property_label}.{orientation}.transE'
label = 'Yval'
print("Summary of class variable: \n", train_data[label].describe())
predictor = TabularPredictor(label=label, path=save_path).fit(train_data, hyperparameters={'NN':{},'GBM':{},'XT':{},'KNN':{}})

In [ ]:
# get best predictor and show leaderboard
# save_path = f'../embeddings/models/agModels.{property_label}.{orientation}.transE.synchronized.250000'
# predictor = TabularPredictor.load(save_path)
y_test = test_data[label]
test_data_nolab = test_data.drop(columns=[label])
print(predictor.get_model_best())
y_pred = predictor.predict(test_data_nolab)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)
predictor.leaderboard(test_data, silent=True)